In [1]:
! pip install -q deepdoctection
! pip install -q paddleocr
! pip install -q paddlepaddle

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.1 which is incompatible.
conda 24.5.0 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
featuretools 1.31.0 requires tqdm>=4.66.3, but you have tqdm 4.64.0 which is incompatible.
fitter 1.7.0 requires tqdm<5.0.0,>=4.65.1, but you have tqdm 4.64.0 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
# 定义全局变量
image_width = 0  # 待处理图片宽度
image_height = 0  # 待处理图片高度
text_size_ocr = 17  # 标准字体大小

class ocr_box:
    txt = ''
    box = None

    def __init__(self, box, txt):
        self.box = box
        self.txt = txt


class layout_box:
    txt = ''
    ocr_box_list = []
    ocr_box_list_size = 0
    Bbox = None

    def __init__(self, box, list):
        self.Bbox = box
        self.ocr_box_list = list

    # 添加OCR box到res下
    def append_ocr_box(self, box):
        self.ocr_box_list.append(box)
        self.txt = self.txt + box.txt + ' ' 
        self.ocr_box_list_size += 1
        return self.ocr_box_list_size

    def is_true_text(self):
        global image_width
        if image_width > 0:
            max_margin = image_width // 5
        else:
            max_margin = 200
        x_list = []  # 获取每一个list的X0
        
        if self.ocr_box_list_size >= 2:  # 条件一：必须有两个box及以上
            for idx, b in enumerate(self.ocr_box_list):  # 获取每一个b的(x0)
                x_list.append(b.box[0])  # 实际[x0,y0,x1，y1]坐标系要对应，获得x[0]坐
            for x in x_list:
                for y in x_list:
                    # 此处对相对距离进行量化，如果Layout内每一句OCRbox开始坐标点相差过大，则该text不是一个自然段的
                    if abs(x - y) >= max_margin:  # 像素大小需要根据文档长度进行量化
                        return False  # 条件二：X轴值相差超过
        else:
            return False  # 如果只包含一个文本则直接返回false
        return True
    
    def is_true_layout(self):
        global image_width
        if image_width > 0:
            edge_margin = image_width * 4 / 5 
        else:
            edge_margin = 1365 # temp value
        if self.ocr_box_list_size >= 2:
            for idx, b in enumerate(self.ocr_box_list):
                if b.box[2] >= edge_margin: return True
        return False
            
            
# 工具类================================================================================================================

import math
import cv2
from PIL import ImageFont, Image, ImageDraw
import numpy as np
import datetime
import os
import glob
import requests
import random
import json
from hashlib import md5


# 防止字符串乱码
os.environ['NLS_LANG'] = 'SIMPLIFIED CHINESE_CHINA.UTF8'

# 绘制字体函数
def draw_box_txt_fine(img_size, txt, font_path="/kaggle/input/font-ttf/fonts/simfang.ttf", is_multiline=True, box=None,
                      multi_box=None, text_size_ocr=25):  # multi_box(width,height)
    def create_font(txt, sz, font_path="/kaggle/input/font-ttf/fonts/simfang.ttf", text_size_ocr=25, is_multiline=False):  # 正常【width,height】
        # font_size = int(sz[1] * 0.85)  # 获得宽，中文文本可在此处调节
        is_overflow = False  # 是否越界
        if is_multiline:
            font_size = text_size_ocr  # 暂定正文文本为25
            font = ImageFont.truetype(font_path, font_size, encoding="utf-8")
            length = font.getlength(txt)
        else:
            font_size = round(sz[1] * 0.85)
            # 对字体大小进行量化
            font_size = font_size - (font_size % 3)  # 3倍量化
            if 10 < font_size < 36:
                font_size = text_size_ocr
            font = ImageFont.truetype(font_path, font_size, encoding="utf-8")
            length = font.getlength(txt)
        first_line = True
        # 存储当前句子
        space_txt = ''

        if not is_multiline and length >= sz[0]:  # 如果只有一行,且改行不是文本段洛
            font_size = int(font_size * sz[0] / length)  # 得到一个相对大小的比例（长*款/字长） # 出现字体非常非常小的情况时候 一般只适用于英文文本
            sz = [int(sz[0] * 1.5), int(sz[1])]  # [height, width]
            font = ImageFont.truetype(font_path, font_size, encoding="utf-8")
            is_overflow = True
            space_txt += txt
            return font, sz, space_txt, is_overflow  # 多返回值没问题

        if is_multiline:
            while str_count(txt)[0] >= int(sz[0]) // font_size:
                if str_count(txt)[0] == 0 or int(sz[0]) // font_size == 0: break;
                temp = txt
                txt = temp[:int(sz[0]) // font_size]  # 此处有问题，需要返回被省略的标准函数
                alpha_temp = str_count(txt)[1]
                # 此处重新调整换行判断
                if alpha_temp > 2:
                    idx = (int(sz[0]) // font_size) + alpha_temp
                    txt = temp[:idx]
                    space_txt += txt + '\n'
                    txt = temp[idx:]
                    continue
                else:
                    txt = txt

                # 重新统计txt句子里的标点符号，并经行重新阶段
                if first_line:
                    new_txt = '' + txt  # 首行缩进
                    first_line = False
                else:
                    new_txt = txt  # 在此处确认是否要缩进
                space_txt += new_txt + '\n'
                txt = temp[int(sz[0]) // font_size:]
        space_txt += txt

        return font, sz, space_txt, is_overflow  # 多返回值没问题

    def str_count(s):
        import string
        count_zh = count_pu = count_punc = 0
        s_len = len(s)
        en_dg_count = 0
        for c in s:
            if c in string.ascii_letters or c.isdigit() or c.isspace():  # 此处没有计算标点符号
                en_dg_count += 1
            elif c.isalpha():
                count_zh += 1
            elif c in string.punctuation:
                count_punc += 1
            else:
                count_pu += 1
        # 返回当前句子长度，以及标点符号。
        return s_len - math.ceil((en_dg_count + count_punc) * 0.5), math.ceil((en_dg_count + count_punc) * 0.5)

    if box:
        box_height = int(
            math.sqrt((box[0][0] - box[3][0]) ** 2 + (box[0][1] - box[3][1]) ** 2))  # height
        box_width = int(
            math.sqrt((box[0][0] - box[1][0]) ** 2 + (box[0][1] - box[1][1]) ** 2))  # width
    else:
        box_height = multi_box[3]
        box_width = multi_box[2]

    if box_height > 2 * box_width and box_height > 30:  # 垂直文本框
        img_text = Image.new('RGB', (box_height, box_width), (255, 255, 255))  # 白色文本
        draw_text = ImageDraw.Draw(img_text)
        if txt:
            # 对中文字体经行缩小 如果需要输出英文文档需要重新更改
            font, sz, space_txt, is_overflow = create_font(txt, (box_height, box_width), font_path)
            draw_text.multiline_text([0, 0], space_txt, fill=(0, 0, 0), font=font)  # 从(0,0）开始绘制
        img_text = img_text.transpose(Image.ROTATE_270)  # 进行图像旋转270°？
    else:
        if is_multiline:
            img_text = Image.new('RGB', (box_width, box_height), (255, 255, 255))
            draw_text = ImageDraw.Draw(img_text)
            if txt:
                font, sz, space_txt, is_overflow = create_font(txt, (box_width, box_height),
                                                               font_path, is_multiline=is_multiline,
                                                               text_size_ocr=text_size_ocr)
                draw_text.multiline_text([0, 0], space_txt, fill=(0, 0, 0), font=font)  # 从(0,0）开始绘制
        else:
            img_text = Image.new('RGB', (box_width, box_height), (255, 255, 255))
            draw_text = ImageDraw.Draw(img_text)
            if txt:
                font, sz, space_txt, is_overflow = create_font(txt, (box_width, box_height), font_path)
                if is_overflow:
                    img_text = Image.new('RGB', (sz[0], sz[1]), (255, 255, 255))  # 重新绘图
                    draw_text = ImageDraw.Draw(img_text)
                    draw_text.multiline_text([0, 0], space_txt, fill=(0, 0, 0), font=font)  # 从(0,0）开始绘制
                else:
                    draw_text.multiline_text([0, 0], space_txt, fill=(0, 0, 0), font=font)  # 从(0,0）开始绘制

    # 以下使用CV对图像进行透视变换
    pts1 = np.float32(
        [[0, 0], [box_width, 0], [box_width, box_height], [0, box_height]])  # 源标准位置box
    if box:
        pts2 = np.float32(
            [box[0], [box_width + box[0][0], box[0][1]], [box_width + box[0][0], box_height + box[0][1]],
             [box[0][0], box_height + box[0][1]]])  # 源标准位置box
    else:
        pts2 = np.float32(
            [[multi_box[0], multi_box[1]], [box_width + multi_box[0], multi_box[1]],
             [box_width + multi_box[0], box_height + multi_box[1]],
             [multi_box[0], box_height + multi_box[1]]])  # 源标准位置box
    # pts2 = np.array(box, dtype=np.float32)  # box的bix

    M = cv2.getPerspectiveTransform(pts1, pts2)  # 透视变换的变换矩阵

    img_text = np.array(img_text, dtype=np.uint8)  # 图像
    img_right_text = cv2.warpPerspective(  # 图像的透视变化
        img_text,
        M,
        img_size,  # 需要输出的图像大小
        flags=cv2.INTER_NEAREST,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=(255, 255, 255))

    return img_right_text  # 图片是一个整张大小图片


# 填充图片
def bincount_1(pic_path):
    img = pic_path

    blue_list = []
    green_list = []
    red_list = []

    for x in range(img.shape[0]):  # 图片的高
        for y in range(img.shape[1]):  # 图片的宽
            px = img[x, y]  # 获得坐标
            blue_list.append(img[x, y][0])
            green_list.append(img[x, y][1])
            red_list.append(img[x, y][2])
            # print(px)  # 这样就能得到每个点的bgr值
    blue_count = np.bincount(blue_list)
    green_count = np.bincount(green_list)
    red_count = np.bincount(red_list)

    blue = np.argmax(blue_count) + 1
    green = np.argmax(green_count) + 1
    red = np.argmax(red_count) + 1

    return blue, green, red



# 百度翻译
def baidu_translator(text='', from_lang='en', to_lang='zh'):
    # Set your own appid/appkey.
    appid = '20240628002087171'
    appkey = 'uN9S5RZGcBZFjgCmmiYg'

    # For list of language codes, please refer to `https://api.fanyi.baidu.com/doc/21`
    from_lang = from_lang
    to_lang = to_lang
    query = text

    endpoint = 'http://api.fanyi.baidu.com'
    path = '/api/trans/vip/translate'
    url = endpoint + path

    # Generate salt and sign
    def make_md5(s, encoding='utf-8'):
        return md5(s.encode(encoding)).hexdigest()

    salt = random.randint(32768, 65536)
    sign = make_md5(appid + query + str(salt) + appkey)

    # Build request
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    payload = {'appid': appid, 'q': query, 'from': from_lang, 'to': to_lang, 'salt': salt, 'sign': sign}
    # Send request
    r = requests.post(url, params=payload, headers=headers)
    result = r.json()  # json file
    # print(result['trans_result'])
    # for p in result['trans_result']:
    #     print(str(p['dst']))
    return result['trans_result']

def resize_keep_aspectratio(image_src, dst_size):
    src_h, src_w = image_src.shape[:2]
    # print(src_h, src_w)
    dst_h, dst_w = dst_size

    # 判断应该按哪个边做等比缩放
    h = dst_w * (float(src_h) / src_w)  # 按照ｗ做等比缩放
    w = dst_h * (float(src_w) / src_h)  # 按照h做等比缩放

    h = int(h)
    w = int(w)

    if h <= dst_h:
        image_dst = cv2.resize(image_src, (dst_w, int(h)))
    else:
        image_dst = cv2.resize(image_src, (int(w), dst_h))

    h_, w_ = image_dst.shape[:2]
    # print(h_, w_)

    top = int((dst_h - h_) / 2)
    down = int((dst_h - h_ + 1) / 2)
    left = int((dst_w - w_) / 2)
    right = int((dst_w - w_ + 1) / 2)

    value = [0, 0, 0]
    borderType = cv2.BORDER_CONSTANT
    # print(top, down, left, right)
    image_dst = cv2.copyMakeBorder(image_dst, top, down, left, right, borderType, None, value)

    return image_dst

# 工具类结束================================================================================================= 

def is_in(box1, box2):  # 传入的参数(x0,y0,x1,y1)，box1是否在box2里面
    xa0, ya0, xa1, ya1 = box1
    xb0, yb0, xb1, yb1 = box2
    # (a,b)为box1 的质心
    a = (xa1 - xa0) / 2 + xa0
    b = (ya1 - ya0) / 2 + ya0
    # print((a,b))
    # print(box2)
    # 质心在box2围城的区域之中
    if (a >= xb0 and a <= xb1) and (b >= yb0 and b <= yb1):
        return True
    else:
        return False


# 计算图像中标准字体大小,只针对
def average_text_size(ocr_list):
    size_list = []
    temp_size = 0
    global text_size_ocr
    for ocr in ocr_list:
        height = int(abs(ocr.box[3] - ocr.box[1]))
        font_size = round(height * 0.85)
        # 对字体大小进行量化
        font_size = font_size - (font_size % 3)  # 3倍量化
        if 10 < font_size <= 20:
            font_size = 15
        elif 20 < font_size <= 36:
            font_size = 25
        size_list.append(font_size)
    for size in size_list:
        temp_size += size
    size_count = np.bincount(size_list)
    # 跟新全局变量
    text_size_ocr = np.argmax(size_count) + 1
    
def rectangle_normalization_bbox(box):
    box_height = int(
        math.sqrt((box[0][0] - box[3][0]) ** 2 + (box[0][1] - box[3][1]) ** 2))  # 高
    box_width = int(
        math.sqrt((box[0][0] - box[1][0]) ** 2 + (box[0][1] - box[1][1]) ** 2))  # 长
    x0 = int(box[0][0])
    x1 = int(box[0][0] + box_width)
    y0 = int(box[0][1])
    y1 = int(box[0][1] + box_height)

    return x0, y0, x1, y1  # 返回一个标准化的bbox


def draw_ocr_box_txt_one_pic(image,
                             boxes,
                             txts,
                             page=None,
                             scores=None,
                             drop_score=0.8,  # 准确度门限
                             font_path="/kaggle/input/font-ttf/fonts/simfang.ttf"):
    # 全局变量更新
    global image_width, image_height
    image_height = image.height
    image_width = image.width
    h, w = image.height, image.width
    img_left = image.copy()
    doc_text_list = []
    doc_bbox_list = []
    doc_table_bbox_lsit = []
    img_right = np.ones((h, w, 3), dtype=np.uint8) * 255  # 初始化一个同样大小的图片
    random.seed(0)
    origin_result = '' # 存储源翻译文本
    trans_result = None # 存储翻译结果的字典

    # ImageDraw模块提供了图像对象的简单2D绘制。用户可以使用这个模块创建新的图像，注释或润饰已存在图像，为web应用实时产生各种图形。 Draw()创建一个可以在给定图像上绘图的对象。
    # img_left_array = np.array(img_left)  # 分支一
    img1_left_cv2 = cv2.cvtColor(np.array(img_left), cv2.COLOR_RGB2BGR)
    draw_left = ImageDraw.Draw(img_left)  # 直接进行绘画
    # 在此处处理page
    for layout in page.layouts:
        if layout.category_name=="text":
            doc_text_list.append(layout.text)
            doc_bbox_list.append((int(layout.bounding_box.ulx),int(layout.bounding_box.uly),int(layout.bounding_box.lrx),
                                 int(layout.bounding_box.lry))) # 返回一个坐标元组
    for table in page.tables:
        doc_table_bbox_lsit.append(table.bbox)
        print(f'这是tableBox：{table.bbox}')
    bbox_codin = doc_bbox_list
    layout_list = []  # 传入的参数(x0,y0,x1,y1)
    ocr_list = []  # 用于存储需要进行绘制的OCR
    # 初始化layout_list
    for layout in bbox_codin:
        ocr_save_list = []  # 防止List驻留导致的地址相同恶行Bug
        layout_instance = layout_box(layout, ocr_save_list)
        layout_list.append(layout_instance)
    # 进行一次便利，将OCR_list 和 Layout_list 进行初始化，进行标准字体确定和margin确定
    for idx, (box, txt) in enumerate(zip(boxes, txts)):
        if scores is not None and scores[idx] < drop_score:  # 过滤准确度低的文本信息
            continue
        x0,y0,x1,y1 = rectangle_normalization_bbox(box)  #  return y0, y1, x0, x1  # 返回一个标准化的bbox
        ocr_box_instance = ocr_box((x0, y0, x1, y1), txt)  # 创建一个实例【y0,x0,y1,x1】->实际[x0,y0,x1，y1]坐标系要对应
        ocr_list.append(ocr_box_instance)  # 将实例0入列表
        for layout in layout_list:
            if is_in(ocr_box_instance.box, layout.Bbox):
                # 如果该OCRBOX在Layoutbox内,加入到子类
                layout.append_ocr_box(ocr_box_instance)  # 一次匹配成功后所有的layout都添加了该类？？
                ocr_list.pop(-1)  # 弹出该ocrbox实例，此处正确
                break
    # 统计字体大小
    average_text_size(ocr_list)
    #选取需要的Layout_list_temp，用于处理自然段文本需要
    layout_list_temp = []  # 用与存取正确的layout
    for layout in layout_list:
        if layout.is_true_text() and layout.is_true_layout():  # 该函数是自然段
            # print("是自然段文本，且内部OCR长度为：" + str(layout.ocr_box_list_size))
            print('Got one')
            layout_list_temp.append(layout)
        else:
            # 需要在Ocrlist里面push其内部的OCR
            for ocr in layout.ocr_box_list:
                ocr_list.append(ocr)
    # 得到的layout_list_temp[] 用于文本段的绘制 和 ocr_list，用于剩下需要单独绘制的OCR
    # 此处需要对ocr_list 和layout_list_temp[]集中处理
    # TODO : 将每一句txt进行以'\n'的打包，传入baidu翻译进行通篇翻译，返回dict结果存在trans_result[]里面
    index = 0
    for o in ocr_list:
        origin_result += o.txt + '\n'
        index+=1
    for layout_t in layout_list_temp:
        origin_result += layout_t.txt + '\n'
        # print('这是txt中的内容：'+layout_t.txt)

    trans_result = baidu_translator(text=origin_result)
    # for p in trans_result:
    #     print('翻译：'+str(p['src']))
    # 填涂背景与OCR的书写
    for idx, (box, txt) in enumerate(zip(boxes, txts)):
        if scores is not None and scores[idx] < drop_score:  # 过滤准确度低的文本信息
            continue
        # 填充白色低（但想根据其背景进行识别），需要获取原图片，再把原图片经行填充
        # TODO
        # 绘图的时候将bbox转为对应的标准矩形
        box_reg = rectangle_normalization_bbox(box)  # X0
        # 截取该矩形图片
        img1_left_cv2_temp = img1_left_cv2[box_reg[1]:box_reg[3], box_reg[0]:box_reg[2]]  # 注意此处是【h,w】

        blue, green, red = bincount_1(img1_left_cv2_temp)
#         draw_left.polygon(
#             [
#                 box[0][0], box[0][1], box[1][0], box[1][1], box[2][0],
#                 box[2][1], box[3][0], box[3][1]
#             ],
#             fill=(red, green, blue),  # (red,green,blue)
#             outline=None)

        # 当前ocr的[x0,y0]是否在单句list里面
        table_num = 0
        for ocr in ocr_list:
            # 如果在，则正常绘制
            if (ocr.box[0] == box_reg[0]) and (ocr.box[1] == box_reg[1]):
                # 填字的时候仍然使用的是曲边box
                # TODO 此处要对txt进行筛选
                for re in trans_result:
                    if re['src'] == txt:
                        txt = re['dst'] # 此处进行文本替换
                        break
                
                if len(doc_table_bbox_lsit) > 0 :
                    for table in  doc_table_bbox_lsit:
                        if is_in(ocr.box, table):
                            table_num +=1
                            
                if table_num ==0 :
                    img_right_text = draw_box_txt_fine((w, h), txt, font_path, box=box)  # 返回的是一个个包含文本信息的图片，整张图片的相对位置
                        # 此处进行对修改后的副本经行文字融合
                    img_right = cv2.bitwise_and(img_right, img_right_text)  # 右图位图迭代合并拼接
                    draw_left.polygon(
                        [
                            box[0][0], box[0][1], box[1][0], box[1][1], box[2][0],
                            box[2][1], box[3][0], box[3][1]
                        ],
                        fill=(red, green, blue),  # (red,green,blue)
                        outline=None)
                break # 找到了处理完
        if box_reg not in ocr_list and table_num == 0:
            draw_left.polygon(
                        [
                            box[0][0], box[0][1], box[1][0], box[1][1], box[2][0],
                            box[2][1], box[3][0], box[3][1]
                        ],
                        fill=(red, green, blue),  # (red,green,blue)
                        outline=None)
                    
    # 此处单独利用layout_list[]绘制段落
    for layout in layout_list_temp:
        width = layout.Bbox[2] - layout.Bbox[0]
        height = layout.Bbox[3] - layout.Bbox[1]
        layout.txt = trans_result[index]['dst']
        index += 1 # 此处写法不规范，容易造成数组溢出

        # print("这是laoyout文本"+ layout.txt)
        img_right_text = draw_box_txt_fine((w, h), layout.txt, font_path, is_multiline=True,
                                           multi_box=(
                                               layout.Bbox[0], layout.Bbox[1], width,
                                               height))  # 返回的是一个个包含文本信息的图片，整张图片的相对位置
        img_right = cv2.bitwise_and(img_right, img_right_text)  # 右图位图迭代合并拼接

    img_right = Image.fromarray(img_right)
    con_enhancer = ImageEnhance.Contrast(img_right)
    img_right = con_enhancer.enhance(1.1)  # 对比度增强
    # img = Image.fromarray(cv2.cvtColor(img1_left_cv2, cv2.COLOR_BGR2RGB))
    img = img_left
    # 新建一张图，设置RGB色彩(255, 255, 255)，设置宽度和高度，这里设置2倍的宽是为了粘贴2张图像
    img_show = Image.new('RGB', (w * 2, h), (255, 255, 255))
    # 把图像粘贴在img_show上
    img_show.paste(img, (0, 0, w, h))
    img_show.paste(img_right, (w, 0, w * 2, h))

    # # 图片合并
    box_1 = (0, 0, w, h)
    box_2 = (w, 0, 2 * w, h)
    img_left = img_show.crop(box_1)
    img_right = img_show.crop(box_2)
    img_merge = Image.blend(img_left, img_right, 0.5)
    con_enhancer = ImageEnhance.Contrast(img_merge)
    img_merge = con_enhancer.enhance(2.5)  # 对比度增强
    # 此处进行图像缩小

    img_merge = cv2.cvtColor(np.asarray(img_merge), cv2.COLOR_RGB2BGR)
    scale_num = 0.5 # 缩小系数为.0.5
    dist_size = (int(scale_num * img_merge.shape[0]), int(scale_num * img_merge.shape[1]))
    cv_image = resize_keep_aspectratio(img_merge, dist_size)  # 对文件进行等缩小
    img_merge = Image.fromarray(cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB))

    return img_merge  # 返回Image对象
    # return img_show


In [3]:
import deepdoctection as dd

import cv2
from pathlib import Path
from matplotlib import pyplot as plt
from IPython.core.display import HTML
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageEnhance
import cv2
import pandas as pd
from paddleocr import PaddleOCR, draw_ocr

2024-07-08 05:55:04.399954: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 05:55:04.400155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 05:55:04.563772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[0708 05:55.15 @utils.py:161]  INF  NumExpr defaulting to 4 threads.
[0708 05:55.18 @__init__.py:433]  WRN  Both DD_USE_TORCH and DD_USE_TF are set. Defaulting to PyTorch. If you want a different behaviour, set DD_USE_TORCH to None before importing deepdoctection.


In [4]:
analyzer = dd.get_dd_analyzer()
match_service = analyzer.pipe_component_list[6]
match_service.max_parent_only = True
text_order_service = analyzer.pipe_component_list[7]
text_order_service.include_residual_text_container = False
ocr = PaddleOCR(use_angle_cls=True, lang="en",use_gpu=False, show_log=False)  
file_path = '/kaggle/input/text-pic-for-table/'
save_folder = '/kaggle/working/'
df = analyzer.analyze(path=file_path)
df.reset_state()
doc=iter(df)
pictures = os.listdir(path=file_path)
for pic in pictures:
    page = next(doc)
    dic_path = file_path + page.file_name
#     page = next(doc) # 此处doc和pic的迭代方式不一样
    image = Image.open(dic_path).convert('RGB')  # 打开图片 Image 格式
    result = ocr.ocr(dic_path, cls=True)
    boxes = [line[0] for line in result[0]]
    txts = [line[1][0] for line in result[0]]
    scores = [line[1][1] for line in result[0]]
    im_show = draw_ocr_box_txt_one_pic(image, boxes, txts, page, scores, drop_score=0.90,
                                           font_path='/kaggle/input/font-ttf/fonts/simfang.ttf') # 此处确定置信度区间为0.85
    path = save_folder + page.file_name
    im_show.save(path)
    print(path + '处理完成！')


d2_model_0829999_layout_inf_only.ts:   0%|          | 0.00/275M [00:00<?, ?B/s]

[0708 05:55.27 @model.py:1021]  ERR  File downloaded from deepdoctection/d2_casc_rcnn_X_32xd4_50_FPN_GN_2FC_publaynet_inference_only does not match the expected size! You may have downloaded a broken file, or the upstream may have modified the file.


CASCADE_RCNN_R_50_FPN_GN_TS.yaml:   0%|          | 0.00/143 [00:00<?, ?B/s]

d2_model_1639999_item_inf_only.ts:   0%|          | 0.00/275M [00:00<?, ?B/s]

[0708 05:55.33 @model.py:1021]  ERR  File downloaded from deepdoctection/d2_casc_rcnn_X_32xd4_50_FPN_GN_2FC_pubtabnet_rc_inference_only does not match the expected size! You may have downloaded a broken file, or the upstream may have modified the file.


...

download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:05<00:00, 1.98MiB/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:00<00:00, 4.60MiB/s]
|          |11/?[00:00<00:00,1285.23it/s]


/kaggle/working/images_10.png处理完成！
Got one
/kaggle/working/images_11.png处理完成！
Got one
Got one
Got one
Got one
Got one
/kaggle/working/images_15.png处理完成！
Got one
Got one
Got one
Got one
Got one
Got one
Got one
/kaggle/working/images_16.png处理完成！
Got one
Got one
Got one
Got one
Got one
Got one
Got one
Got one
/kaggle/working/images_17.png处理完成！
Got one
Got one
Got one
Got one
Got one
Got one
/kaggle/working/images_18.png处理完成！
Got one
Got one
Got one
Got one
Got one
Got one
Got one
Got one
/kaggle/working/images_5.png处理完成！
这是tableBox：[285.0, 868.0, 1127.0, 1247.0]
这是tableBox：[294.0, 297.0, 1499.0, 770.0]
/kaggle/working/images_6.png处理完成！
这是tableBox：[279.0, 419.0, 1576.0, 593.0]
这是tableBox：[218.0, 1387.0, 1541.0, 2048.0]
Got one
/kaggle/working/images_7.png处理完成！
Got one
Got one
/kaggle/working/images_8.png处理完成！
Got one
Got one
Got one
Got one
/kaggle/working/images_9.png处理完成！
